In [21]:
import cv2, math
import imutils
import numpy as np
from sklearn.metrics import pairwise
import vlc, time

bg = None
media = vlc.MediaPlayer(r"C:\Users\Vicky Kumar\Desktop\videohandler\Emiway ft. Macklemore - Firse Machayenge.mkv")
media.play()

def run_avg(image, accumWeight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image, bg, accumWeight)

def segment(image, threshold=25):
    global bg
    diff = cv2.absdiff(bg.astype("uint8"), image)
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)
    
def mode(test_list):
    max = 0
    res = test_list[0] 
    for i in test_list: 
        freq = test_list.count(i) 
        if freq > max: 
            max = freq 
            res = i
    return res
    
def count(thresholded, segmented):
    chull = cv2.convexHull(segmented)
    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])
    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]
    radius = int(0.8 * maximum_distance)
    circumference = (2 * np.pi * radius)
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    ( cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1
    return count

if __name__ == "__main__":
    accumWeight = 0.5
    camera = cv2.VideoCapture(0)
    top, right, bottom, left = 10, 350, 225, 590
    num_frames = 0
    calibrated = False
    avg = []
    
    while(True):
        (grabbed, frame) = camera.read()
        frame = imutils.resize(frame, width=700)
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)
        
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            hand = segment(gray)
            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                fingers = count(thresholded, segmented)
                avg.append(fingers)
                res = mode(avg) 
                print(res)
                if fingers==0 or fingers==1:
                    avg = []
                if res>3:
                    media.pause()
                    time.sleep(.001)
                cv2.putText(clone, str(res), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                cv2.imshow("Thesholded", thresholded)
                
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)
        num_frames += 1
        cv2.imshow("Video Feed", clone)
        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord("q"):
            break
            
media.stop()
camera.release()
cv2.destroyAllWindows()

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...
2


In [18]:
avg.append(fingers)
print(math.ceil(mean(avg)))

2


In [19]:
avg.append(fingers)
res = statistics.mode(avg) 
print(res)

2


In [22]:
def mode(test_list):
    max = 0
    res = test_list[0] 
    for i in test_list: 
        freq = test_list.count(i) 
        if freq > max: 
            max = freq 
            res = i
    return res

mode([1,4,2,7,7,7,7,7,7,77,3,4,4])

7

In [14]:
import vlc
media = vlc.MediaPlayer(r"C:\Users\Vicky Kumar\Desktop\videohandler\Emiway ft. Macklemore - Firse Machayenge.mkv")

In [17]:
media.play()

0

In [16]:
media.pause()

In [18]:
media.stop()